# INSTALLING DEPENDENCIES
(Might need to restart runtime to correctly install textattack)

In [ ]:
!pip install textattack
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt
from textattack.datasets import HuggingFaceDataset
from textattack.attack_recipes import PWWSRen2019
from textattack import Attacker
import textattack

import numpy as np
import torch


from textattack.models.wrappers import ModelWrapper

import pandas as pd
import codecs


print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print(
    "GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE"
)


Version:  2.14.0
Eager mode:  True
Hub version:  0.15.0
GPU is NOT AVAILABLE


#READING INPUT FILE

In [ ]:
with codecs.open('spam.csv', 'r', encoding='ISO-8859-1') as f:
    df = pd.read_csv(f)


data = df.values.tolist()
dataclean = []
for example in data:
  if example[0] == "ham":
    dataclean.append((example[1],0))
  if example[0] == "spam":
    dataclean.append((example[1],1))



train_data = dataclean[0:4000]
test_data = dataclean[4000:5000]

train_examples, train_labels = np.array(train_data)[:,0],np.array(train_data)[:,1]
test_examples, test_labels = np.array(test_data)[:,0],np.array(test_data)[:,1]

adversary_data = dataclean[5000:]


#TRAINING MODEL TO CLASSIFY SMS AS SPAM

In [ ]:
#loading embedding model form tenserflow dataset
model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(
    model, output_shape=[20], input_shape=[], dtype=tf.string, trainable=True
)

## adding a layer of classifier on top of embedding layer
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation="relu"))
model.add(tf.keras.layers.Dense(1))

model.summary()

x_val = test_examples
partial_x_train = train_examples

y_val = test_labels.astype(np.float)
partial_y_train = train_labels.astype(np.float)

model.compile(
    optimizer="adam",
    loss=tf.losses.BinaryCrossentropy(from_logits=True),
    metrics=["accuracy"],
)


# training model on loaded dataset
history = model.fit(
    partial_x_train,
    partial_y_train,
    epochs=30,
    batch_size=512,
    validation_data=(x_val, y_val),
    verbose=1,
)

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_5 (KerasLayer)  (None, 20)                400020    
                                                                 
 dense_10 (Dense)            (None, 16)                336       
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 400373 (1.53 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30


<ipython-input-51-3e94313d62fa>:33: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_val = test_labels.astype(np.float)
<ipython-input-51-3e94313d62fa>:34: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  partial_y_train = train_labels.astype(np.float)


8/8 [==============================] - 2s 69ms/step - loss: 0.7681 - accuracy: 0.7387 - val_loss: 0.6463 - val_accuracy: 0.8350
Epoch 2/30
8/8 [==============================] - 0s 28ms/step - loss: 0.5816 - accuracy: 0.8690 - val_loss: 0.5009 - val_accuracy: 0.8830
Epoch 3/30
8/8 [==============================] - 0s 28ms/step - loss: 0.4652 - accuracy: 0.8900 - val_loss: 0.4121 - val_accuracy: 0.9020
Epoch 4/30
8/8 [==============================] - 0s 34ms/step - loss: 0.3940 - accuracy: 0.8978 - val_loss: 0.3576 - val_accuracy: 0.9040
Epoch 5/30
8/8 [==============================] - 0s 31ms/step - loss: 0.3489 - accuracy: 0.9013 - val_loss: 0.3226 - val_accuracy: 0.9090
Epoch 6/30
8/8 [==============================] - 0s 29ms/step - loss: 0.3188 - accuracy: 0.9045 - val_loss: 0.2982 - val_accuracy: 0.9150
Epoch 7/30
8/8 [==============================] - 0s 29ms/step - loss: 0.2962 - accuracy: 0.9100 - val_loss: 0.2799 - val_accuracy: 0.9180
Epoch 8/30
8/8 [======================

#RUNNNIG ATTACK ON TRAINED MODEL

In [ ]:
# declaring custom model wrapper to ensure output format dependencies
class CustomTensorFlowModelWrapper(ModelWrapper):
    def __init__(self, model):
        self.model = model

    def __call__(self, text_input_list):
        text_array = np.array(text_input_list)
        preds = self.model(text_array).numpy()
        logits = torch.exp(-torch.tensor(preds))
        logits = 1 / (1 + logits)
        logits = logits.squeeze(dim=-1)
        # Since this model only has a single output (between 0 or 1),
        # we have to add the second dimension.
        final_preds = torch.stack((1 - logits, logits), dim=1)
        return final_preds

In [ ]:

model_wrapper = CustomTensorFlowModelWrapper(model)

attack_args = textattack.AttackArgs(
    num_examples=40
)

dataset = textattack.datasets.Dataset(adversary_data)
attack = PWWSRen2019.build(model_wrapper)
attacker = Attacker(attack, dataset,attack_args)
attacker.attack_dataset()


[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
textattack: Unknown if model of class <class 'keras.src.engine.sequential.Sequential'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.


Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  weighted-saliency
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapWordNet
  (constraints): 
    (0): RepeatModification
    (1): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 1 / 1 / 0 / 2:   5%|▌         | 2/40 [00:00<00:02, 13.07it/s]

--------------------------------------------- Result 1 ---------------------------------------------
[[0 (97%)]] --> [[[FAILED]]]

Nope. Since ayo travelled, he has forgotten his guy


--------------------------------------------- Result 2 ---------------------------------------------
[[0 (99%)]] --> [[1 (64%)]]

You [[still]] [[around]]? Looking to pick up [[later]]

You [[static]] [[approximately]]? Looking to pick up [[recent]]




[Succeeded / Failed / Skipped / Total] 2 / 1 / 0 / 3:   8%|▊         | 3/40 [00:00<00:04,  7.68it/s]

...

--------------------------------------------- Result 34 ---------------------------------------------
[[0 (98%)]] --> [[1 (81%)]]

[[I]] want to [[go]] to perumbavoor

[[1]] want to [[X]] to perumbavoor


--------------------------------------------- Result 35 ---------------------------------------------
[[0 (99%)]] --> [[[FAILED]]]

How many times i told in the stage all use to laugh. You not listen aha.


--------------------------------------------- Result 36 ---------------------------------------------
[[0 (99%)]] --> [[[SKIPPED]]]

You won't believe it but it's true. It's Incredible Txts! Reply G now to learn truly amazing things that will blow your mind. From O2FWD only 18p/txt


--------------------------------------------- Result 37 ---------------------------------------------
[[0 (98%)]] --> [[[FAILED]]]

(You didn't hear it from me)




[Succeeded / Failed / Skipped / Total] 19 / 16 / 3 / 38:  95%|█████████▌| 38/40 [00:05<00:00,  7.45it/s]

--------------------------------------------- Result 38 ---------------------------------------------
[[0 (100%)]] --> [[[FAILED]]]

Thanks for being there for me just to talk to on saturday. You are very dear to me. I cherish having you as a brother and role model.


--------------------------------------------- Result 39 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 20 / 16 / 4 / 40: 100%|██████████| 40/40 [00:05<00:00,  7.52it/s]

[[0 (98%)]] --> [[1 (68%)]]

Pls clarify [[back]] if an open return ticket that i have can be preponed for me to [[go]] [[back]] to kerala.

Pls clarify [[stake]] if an open return ticket that i have can be preponed for me to [[X]] [[stake]] to kerala.


--------------------------------------------- Result 40 ---------------------------------------------
[[0 (59%)]] --> [[[SKIPPED]]]

Natalie (20/F) is inviting you to be her friend. Reply YES-165 or NO-165 See her: www.SMS.ac/u/natalie2k9 STOP? Send STOP FRND to 62468



+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 20     |
| Number of failed attacks:     | 16     |
| Number of skipped attacks:    | 4      |
| Original accuracy:            | 90.0%  |
| Accuracy under attack:        | 40.0%  |
| Attack success rate:          | 55.56% |
| Average perturbed word %:     | 30.33% |
| Average num. words per input: | 16.4   |
